In [2]:
import dill as dill
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from warnings import filterwarnings

from sklearn.pipeline import Pipeline, FeatureUnion

filterwarnings("ignore")

In [3]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [5]:
X = df.drop('Outcome', axis = 1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 3)

In [13]:
X_test.to_csv('X_test.csv', index = None, encoding='utf-8', header='true')
y_test.to_csv('y_test.csv', index = None, encoding='utf-8', header='true')

In [ ]:
class NumberSelector(BaseEstimator, TransformerMixin):
    # Функция, выбирающая каждый числовой признак датасета
    # для дополнительных преобразований

    def __init__(self, key):
        self.key = key

    def fit(self, X, y = None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [ ]:
final_transformers = list()

for cont_col in list(X):
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key = cont_col))
            ])
    final_transformers.append((cont_col, cont_transformer))

feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [ ]:
pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state=42)),
])

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
with open('randfor_pipeline.dill', 'wb') as f:
    dill.dump(pipeline, f)